In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy import stats
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, ShuffleSplit
from sklearn.preprocessing import *
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFE, RFECV
from sklearn.linear_model import *
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import resample
from sklearn.datasets import make_regression
from sklearn.metrics import *
from sklearn.compose import make_column_transformer
from sklearn.pipeline import Pipeline, make_pipeline
from xgboost import XGBClassifier, XGBRegressor
from imblearn.datasets import make_imbalance
import statsmodels.api as sm
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping

In [42]:
def upsample_classes(data, target):
    
    lst = list(data[target].unique())
    
    classes = []
    for c in lst:
        classes.append(data[data[target]==c])
    
    length = 0
    class_lab = None
    for c in classes:
        if len(c)>length:
            length=len(c)
            class_lab = c
    class_lab = class_lab[target].unique()[0]
    
    regroup = pd.concat(classes)
    maj_class = regroup[regroup[target]==class_lab]

    lst.remove(class_lab)
    
    new_classes=[]
    for i in lst:
        new_classes.append(resample(data[data[target]==i],replace=True, n_samples=len(maj_class)))

    minority_classes = pd.concat(new_classes)
    upsample = pd.concat([regroup[regroup[target]==class_lab],minority_classes])

    return upsample

# continuous first

In [20]:
df = pd.read_csv('diamonds.csv')

In [21]:
df.drop('Unnamed: 0',axis=1,inplace=True)

In [22]:
df.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [23]:
df['volume'] = df.x*df.y*df.z
df.drop(['x','y','z'],axis=1,inplace=True)

In [24]:
df.head()

,carat,cut,color,clarity,depth,table,price,volume
0,0.23,Ideal,E,SI2,61.5,55.0,326,38.202030
1,0.21,Premium,E,SI1,59.8,61.0,326,34.505856
2,0.23,Good,E,VS1,56.9,65.0,327,38.076885
3,0.29,Premium,I,VS2,62.4,58.0,334,46.724580
4,0.31,Good,J,SI2,63.3,58.0,335,51.917250


In [7]:
for col in df.select_dtypes(include='O').columns:
    df[col] = LabelEncoder().fit_transform(df[col])
    df[col] = df[col].astype(float)

In [8]:
X = df.drop('price',axis=1)
y = df.price
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=14)

In [9]:
model = Sequential()

n_cols = X_train.shape[1]

model.add(Dense(10, activation = 'relu', input_shape = (n_cols, )))
model.add(Dense(10, activation = 'relu'))
model.add(Dense(1))

model.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [10]:
early_stopping_monitor = EarlyStopping(patience=3)

model.fit(X_train, y_train, validation_split=0.2, epochs = 30, callbacks=early_stopping_monitor)

Epoch 1/30
1012/1012 [==============================] - 1s 969us/step - loss: 13219486.0000 - val_loss: 3952034.2500
Epoch 2/30
1012/1012 [==============================] - 1s 845us/step - loss: 3156731.0000 - val_loss: 2304720.2500
Epoch 3/30
1012/1012 [==============================] - 1s 860us/step - loss: 2699818.5000 - val_loss: 2075834.0000
Epoch 4/30
1012/1012 [==============================] - 1s 859us/step - loss: 2537654.5000 - val_loss: 1948426.6250
Epoch 5/30
1012/1012 [==============================] - 1s 934us/step - loss: 2409607.7500 - val_loss: 1819529.3750
Epoch 6/30
1012/1012 [==============================] - 1s 891us/step - loss: 2303027.5000 - val_loss: 1763979.1250
Epoch 7/30
1012/1012 [==============================] - 1s 959us/step - loss: 2209063.2500 - val_loss: 1660999.8750
Epoch 8/30
1012/1012 [==============================] - 1s 1ms/step - loss: 2148388.2500 - val_loss: 1611302.3750
Epoch 9/30
1012/1012 [==============================] - 1s 1ms/step - los

In [11]:
preds = model.predict(X_test)

In [16]:
print(r2_score(y_test,preds))
print(mean_absolute_error(y_test,preds))

0.9174488168272821
646.9385434810878


In [17]:
model2 = Sequential()

model2.add(Dense(200, activation='relu', input_shape=(n_cols,)))
model2.add(Dense(200, activation='relu'))
model2.add(Dense(200, activation='relu'))
model2.add(Dense(1))

model2.compile(optimizer = 'adam', loss = 'mean_squared_error')

model2.fit(X_train, y_train, validation_split = 0.2, epochs = 30, callbacks = early_stopping_monitor)

Epoch 1/30
1012/1012 [==============================] - 1s 1ms/step - loss: 4049581.2500 - val_loss: 1551677.8750
Epoch 2/30
1012/1012 [==============================] - 1s 1ms/step - loss: 2403917.7500 - val_loss: 1230586.5000
Epoch 3/30
1012/1012 [==============================] - 1s 1ms/step - loss: 1743293.3750 - val_loss: 1116172.8750
Epoch 4/30
1012/1012 [==============================] - 1s 1ms/step - loss: 1356045.0000 - val_loss: 1076939.2500
Epoch 5/30
1012/1012 [==============================] - 1s 1ms/step - loss: 1203700.3750 - val_loss: 1096590.0000
Epoch 6/30
1012/1012 [==============================] - 1s 1ms/step - loss: 1152181.1250 - val_loss: 1101730.0000
Epoch 7/30
1012/1012 [==============================] - 2s 2ms/step - loss: 1144631.6250 - val_loss: 1054575.8750
Epoch 8/30
1012/1012 [==============================] - 1s 1ms/step - loss: 1141142.3750 - val_loss: 1072937.3750
Epoch 9/30
1012/1012 [==============================] - 1s 1ms/step - loss: 1139722.7500

In [18]:
preds2 = model2.predict(X_test)

In [19]:
print(r2_score(y_test,preds2))
print(mean_absolute_error(y_test,preds2))

0.9238938152132937
598.720934139785


# discrete

In [108]:
df2 = pd.read_csv('sample.csv')

In [109]:
df2.head()

,Unnamed: 0,carat,cut,color,clarity,depth,table,price,price_bin,volume
0,0,0.23,Ideal,E,SI2,61.5,55.0,326.0,0.0,38.202030
1,1,0.21,Premium,E,SI1,59.8,61.0,326.0,0.0,34.505856
2,2,0.23,Good,E,VS1,56.9,65.0,327.0,0.0,38.076885
3,3,0.29,Premium,I,VS2,62.4,58.0,334.0,0.0,46.724580
4,4,0.31,Good,J,SI2,63.3,58.0,335.0,0.0,51.917250


In [110]:
df2['price_bin']=pd.cut(df2.price,bins=2,labels=[0,1])

In [111]:
df2.drop(['Unnamed: 0', 'price'], axis=1, inplace=True)

In [112]:
for col in df2.select_dtypes(include='O').columns:
    df2[col] = LabelEncoder().fit_transform(df2[col])

In [113]:
X = df2.drop(['price_bin'],axis=1)
y = df2.price_bin

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=14)

In [114]:
train = pd.concat([X_train,y_train],axis=1)
up = upsample_classes(train, 'price_bin')

In [115]:
X_train = up.drop('price_bin',axis=1)
y_train = up.price_bin

In [136]:
model3 = Sequential()

n_cols_3 = X_train.shape[1]

model3.add(Dense(250, activation='relu', input_shape = (n_cols_3,)))
model3.add(Dense(250, activation='relu'))
model3.add(Dense(250, activation='relu'))
model3.add(Dense(2, activation='sigmoid'))

In [137]:
model3.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])

In [138]:
model3.fit(X_train, y_train, validation_split=0.2, epochs=30, callbacks=early_stopping_monitor)

Epoch 1/30


ValueError: in user code:

    /Users/josephcohen/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    /Users/josephcohen/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/josephcohen/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/josephcohen/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    /Users/josephcohen/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:533 train_step  **
        y, y_pred, sample_weight, regularization_losses=self.losses)
    /Users/josephc

In [139]:
preds3 = model3.predict(X_test)

In [140]:
print(classification_report(y_test,preds3))

ValueError: Classification metrics can't handle a mix of binary and continuous-multioutput targets